In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sqlite3

db_path = '/content/drive/MyDrive/SQLpractice/store_sales.db'
conn = sqlite3.connect(db_path)

In [14]:
# Create a cursor object
cursor = conn.cursor()

In [15]:
# Execute the SQL query to get table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])

customer
item
transactions


Checking if I am accessing the database previously created and populated:

In [16]:
cursor.execute('SELECT * FROM customer LIMIT 5')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['CustomerID', 'Name', 'Surname', 'Loyalty_Discount']
(200000663, 'Leonard', 'Jackson', 0.02)
(200000661, 'Lucas', 'Abraham', 0.0)
(200000662, 'Trevor', 'Skinner', 0.1)
(100000007, 'Dorothy', 'Thomson', 0.07)
(400000180, 'Eric', 'Bower', 0.06)


In [17]:
cursor.execute('SELECT * FROM item LIMIT 5')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['ItemID', 'Description', 'Retail_Price']
(8619953091, 'Pillowcase', 18.96)
(2020397001, "Men's Pajama Set", 23.1)
(4681342313, 'Sheets', 29.9)
(6697166886, 'Coat', 159.8)
(5187373849, 'Short Slip', 12.87)


In [18]:
cursor.execute('SELECT * FROM transactions LIMIT 5')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['TransactionID', 'Timestamp', 'CustomerID', 'ItemID', 'Shipping_State']
(113255, '2016-12-13 18:05:37', 200000663, 8619953091, 'Delaware')
(113254, '2016-12-13 18:05:22', 200000661, 2020397001, 'Louisiana')
(113253, '2016-12-13 18:03:57', 200000662, 4681342313, 'Louisiana')
(113252, '2016-12-13 18:01:45', 100000007, 6697166886, 'Alabama')
(113251, '2016-12-13 18:01:22', 400000180, 6697166886, 'New Hampshire')


In [19]:
# Close cursor object and the database connection
cursor.close()

# **SQL queries**

In [20]:
# Create a cursor object
cursor = conn.cursor()

### Total sales?

In [21]:
cursor.execute('''SELECT COUNT(*) as number_of_sales
                  FROM transactions;''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['number_of_sales']
(3455,)


### Which day has the most sales?

In [22]:
cursor.execute('''SELECT strftime('%Y-%m-%d', Timestamp) as day, COUNT(*) as count
                  FROM transactions
                  GROUP BY day;''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['day', 'count']
('2016-12-10', 288)
('2016-12-11', 1171)
('2016-12-12', 1167)
('2016-12-13', 829)


### How many of each item were sold on each day?
Note: There are products with the same description.

In [23]:
cursor.execute('''SELECT strftime('%Y-%m-%d', transactions.Timestamp) as day, item.ItemID, item.Description, COUNT(item.ItemID) as count
                  FROM transactions
                  INNER JOIN item
                  ON item.ItemID = transactions.ItemID                  
                  GROUP BY day,item.ItemID
                  ORDER BY count DESC;''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['day', 'ItemID', 'Description', 'count']
('2016-12-11', 8713824671, 'Infant 2 Piece Set', 18)
('2016-12-11', 2210750188, 'Blazer', 16)
('2016-12-11', 6962492078, 'Bath Mat', 16)
('2016-12-12', 2963300774, 'Sweatshirt', 16)
('2016-12-12', 6697166886, 'Coat', 16)
('2016-12-11', 6697166886, 'Coat', 15)
('2016-12-11', 9916068026, 'Gowns/Pajamas', 15)
('2016-12-12', 3126356136, 'Formal Dress', 15)
('2016-12-12', 5997626126, 'Coat/Jacket', 15)
('2016-12-12', 8619953091, 'Pillowcase', 15)
('2016-12-13', 3305177499, 'Coveralls (long sleeve)', 15)
('2016-12-11', 1489444777, 'Short Gown', 14)
('2016-12-11', 2430071492, 'Dust Ruffle', 14)
('2016-12-11', 3123824581, 'Sweatpants', 14)
('2016-12-11', 5736433915, 'Mattress Cover', 14)
('2016-12-12', 1284609276, 'Sweater', 14)
('2016-12-12', 1940915493, 'Long Gown', 14)
('2016-12-12', 2430071492, 'Dust Ruffle', 14)
('2016-12-12', 3405268556, 'Handbag', 14)
('2016-12-12', 6182400847, 'Blazer', 14)
('2016-12-11', 2178891798, 'Hand Towel', 13)
('2016-12

### What is the best selling item each day?

In [24]:
cursor.execute('''SELECT day, ItemID, Description, MAX(count) FROM (SELECT strftime('%Y-%m-%d', transactions.Timestamp) as day, item.ItemID as ItemID, item.Description as Description, COUNT(item.ItemID) as count
                  FROM transactions
                  INNER JOIN item
                  ON item.ItemID = transactions.ItemID            
                  GROUP BY day, item.ItemID)
                  
                  GROUP BY day
                  ;''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['day', 'ItemID', 'Description', 'MAX(count)']
('2016-12-10', 2695940506, 'Sweater', 6)
('2016-12-11', 8713824671, 'Infant 2 Piece Set', 18)
('2016-12-12', 2963300774, 'Sweatshirt', 16)
('2016-12-13', 3305177499, 'Coveralls (long sleeve)', 15)


### Who bought Coat and Robe?

In [25]:
cursor.execute('''SELECT transactions.TransactionID, customer.Name as Name, customer.Surname as Surname, item.Description as Description
                  FROM transactions
                  INNER JOIN customer
                  ON customer.CustomerID = transactions.CustomerID
                  INNER JOIN item
                  ON item.ItemID = transactions.ItemID                
                  WHERE Description IN ('Coat','Robe')
                  ORDER BY Name ASC;''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['TransactionID', 'Name', 'Surname', 'Description']
(111369, 'Adam', 'Sharp', 'Robe')
(112305, 'Adrian', 'Wright', 'Coat')
(111108, 'Alexander', 'Hill', 'Coat')
(111600, 'Alexandra', 'Mathis', 'Coat')
(112682, 'Alison', 'Fisher', 'Coat')
(111525, 'Alison', 'Taylor', 'Coat')
(112991, 'Amanda', 'Mackenzie', 'Coat')
(112938, 'Amanda', 'Young', 'Coat')
(111177, 'Amanda', 'Payne', 'Robe')
(110445, 'Amanda', 'King', 'Robe')
(112701, 'Amy', 'Wilkins', 'Coat')
(111489, 'Amy', 'Glover', 'Coat')
(110163, 'Amy', 'Miller', 'Coat')
(109961, 'Amy', 'Wilkins', 'Robe')
(112568, 'Andrea', 'Dickens', 'Coat')
(110343, 'Andrea', 'Newman', 'Coat')
(109828, 'Andrew', 'Wright', 'Coat')
(113122, 'Angela', 'Lawrence', 'Robe')
(113072, 'Anna', 'Lee', 'Coat')
(110278, 'Audrey', 'McGrath', 'Robe')
(112073, 'Bella', 'Vaughan', 'Coat')
(111862, 'Benjamin', 'Ince', 'Coat')
(111513, 'Bernadette', 'Duncan', 'Coat')
(110726, 'Blake', 'Vance', 'Robe')
(110611, 'Blake', 'Vance', 'Coat')
(110218, 'Blake', 'MacDonald', 'Co

### What are the peak selling times?


In [34]:
cursor.execute('''SELECT strftime('%H', Timestamp) as hour, COUNT(*) as sales_count FROM transactions 
                    GROUP BY hour
                    ORDER BY sales_count DESC
                    LIMIT 1;
               ''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['hour', 'sales_count']
('10', 162)


### Average sales per hour?

In [73]:
cursor.execute('''SELECT hour, AVG(total_sales) as average_hourly_sales from(  

                      SELECT strftime('%H', timestamp) as hour, COUNT(*) as total_sales FROM transactions
                           GROUP BY hour, strftime('%Y-%m-%d', timestamp) )
                      GROUP BY hour    
               ''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['hour', 'average_hourly_sales']
('00', 52.0)
('01', 40.333333333333336)
('02', 48.666666666666664)
('03', 50.666666666666664)
('04', 43.0)
('05', 49.333333333333336)
('06', 42.666666666666664)
('07', 47.333333333333336)
('08', 43.666666666666664)
('09', 48.0)
('10', 54.0)
('11', 46.0)
('12', 49.0)
('13', 50.0)
('14', 49.0)
('15', 53.666666666666664)
('16', 47.666666666666664)
('17', 50.0)
('18', 38.0)
('19', 39.666666666666664)
('20', 48.0)
('21', 46.333333333333336)
('22', 51.0)
('23', 51.0)


### Which state makes the most purchases?

In [ ]:
cursor.execute('''SELECT hour, AVG(total_sales) as average_hourly_sales from(  

                      SELECT strftime('%H', timestamp) as hour, COUNT(*) as total_sales FROM transactions
                           GROUP BY hour, strftime('%Y-%m-%d', timestamp) )
                      GROUP BY hour    
               ''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

### Average sales per hour in the state that made the most purchases?

In [82]:
cursor.execute('''SELECT hour, AVG(total_sales) as average_hourly_sales, state FROM 
                      (
                      SELECT strftime('%H', timestamp) as hour, COUNT(*) as total_sales, Shipping_State as state FROM transactions
                           GROUP BY hour, strftime('%Y-%m-%d', timestamp), Shipping_State
                       )
                      WHERE state = (
                        SELECT Shipping_State from Transactions
                          GROUP BY Shipping_State
                          ORDER BY COUNT(Shipping_State) DESC
                          LIMIT 1
                      )
                      GROUP BY hour    
               ''')


# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['hour', 'average_hourly_sales', 'state']
('00', 2.3333333333333335, 'Texas')
('01', 2.6666666666666665, 'Texas')
('02', 3.6666666666666665, 'Texas')
('03', 2.0, 'Texas')
('04', 2.0, 'Texas')
('05', 4.0, 'Texas')
('06', 2.0, 'Texas')
('07', 3.0, 'Texas')
('08', 1.5, 'Texas')
('09', 3.0, 'Texas')
('10', 1.3333333333333333, 'Texas')
('11', 1.3333333333333333, 'Texas')
('12', 2.0, 'Texas')
('13', 1.6666666666666667, 'Texas')
('14', 2.0, 'Texas')
('15', 1.0, 'Texas')
('16', 1.6666666666666667, 'Texas')
('17', 1.5, 'Texas')
('18', 1.6666666666666667, 'Texas')
('19', 2.3333333333333335, 'Texas')
('20', 2.3333333333333335, 'Texas')
('21', 2.0, 'Texas')
('22', 4.0, 'Texas')
('23', 2.0, 'Texas')


### What was the most expensive product sold?


In [84]:
cursor.execute('''SELECT transactions.ItemID as ItemID, item.Description as Description, MAX(item.Retail_Price) as RetailPrice FROM transactions
                  INNER JOIN item
                    ON item.ItemID = transactions.ItemID

               ''')


# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['ItemID', 'Description', 'RetailPrice']
(6697166886, 'Coat', 159.8)



### What was the largest discount? and the smallest? other discounts?


In [89]:
cursor.execute('''SELECT transactions.ItemID as ItemID, item.Description as Description, item.Retail_Price * (customer.Loyalty_Discount) as Discount FROM transactions
                  INNER JOIN item
                    ON item.ItemID = transactions.ItemID
                  INNER JOIN customer
                    ON customer.CustomerID = transactions.CustomerID
                  ORDER BY Discount DESC

               ''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

['ItemID', 'Description', 'Discount']
(6697166886, 'Coat', 14.382)
(6697166886, 'Coat', 14.382)
(6697166886, 'Coat', 14.382)
(6697166886, 'Coat', 14.382)
(6697166886, 'Coat', 14.382)
(6697166886, 'Coat', 14.382)
(1039854547, 'Coat', 13.583000000000002)
(1039854547, 'Coat', 13.583000000000002)
(1039854547, 'Coat', 13.583000000000002)
(1489444777, 'Short Gown', 13.112000000000002)
(1489444777, 'Short Gown', 13.112000000000002)
(1489444777, 'Short Gown', 13.112000000000002)
(1489444777, 'Short Gown', 13.112000000000002)
(6697166886, 'Coat', 12.784)
(6697166886, 'Coat', 12.784)
(6697166886, 'Coat', 12.784)
(6697166886, 'Coat', 12.784)
(6697166886, 'Coat', 12.784)
(6697166886, 'Coat', 12.784)
(1039854547, 'Coat', 12.2247)
(1039854547, 'Coat', 12.2247)
(2210750188, 'Blazer', 11.976)
(2210750188, 'Blazer', 11.976)
(1082405301, 'Blazer', 11.976)
(2210750188, 'Blazer', 11.976)
(1082405301, 'Blazer', 11.976)
(2210750188, 'Blazer', 11.976)
(2210750188, 'Blazer', 11.976)
(9916068026, 'Gowns/Pajama


### 5 people who bought more in quantity? and in value?



In [93]:
cursor.execute('''SELECT customer.CustomerID as ID, customer.Name as Name, customer.Surname as Surname, COUNT(transactions.ItemID) as ItemsBought FROM transactions
                  INNER JOIN item
                    ON item.ItemID = transactions.ItemID
                  INNER JOIN customer
                    ON customer.CustomerID = transactions.CustomerID
                  GROUP BY ID
                  ORDER BY ItemsBought DESC
                  LIMIT 5
               ''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results of people who bought more in quantity
for row in results:
    print(row)


['ID', 'Name', 'Surname', 'ItemsBought']
(100000299, 'Zoe', 'Johnston', 11)
(300000634, 'Heather', 'Mackay', 9)
(200000819, 'Katherine', 'Grant', 9)
(300000597, 'Julian', 'Lyman', 8)
(200000904, 'Ryan', 'Gray', 8)


In [94]:
cursor.execute('''SELECT customer.CustomerID as ID, customer.Name as Name, customer.Surname as Surname, SUM(item.Retail_Price) as ValueBought FROM transactions
                  INNER JOIN item
                    ON item.ItemID = transactions.ItemID
                  INNER JOIN customer
                    ON customer.CustomerID = transactions.CustomerID
                  GROUP BY ID
                  ORDER BY ValueBought DESC
                  LIMIT 5
               ''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results  of people who bought more in value
for row in results:
    print(row)

['ID', 'Name', 'Surname', 'ValueBought']
(100000119, 'Vanessa', 'Lyman', 732.9)
(100000299, 'Zoe', 'Johnston', 698.41)
(400000396, 'Fiona', 'Buckland', 668.14)
(300000634, 'Heather', 'Mackay', 630.9300000000001)
(200000904, 'Ryan', 'Gray', 618.0399999999998)


### How many products did vanessa Lyman buy? How much did he spend in purchases? How much did she save with the discount? what was the day that andrew made the most purchases?

In [105]:
cursor.execute('''SELECT customer.CustomerID as ID, customer.Name as Name, customer.Surname as Surname, COUNT(transactions.ItemID) as ItemsBought, SUM(item.Retail_Price) as Spent, SUM(item.Retail_Price * customer.Loyalty_Discount) as Saved FROM transactions
                  INNER JOIN item
                    ON item.ItemID = transactions.ItemID
                  INNER JOIN customer
                    ON customer.CustomerID = transactions.CustomerID
                  WHERE Name = 'Vanessa' AND Surname = 'Lyman'
               ''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results of people who bought more in quantity
for row in results:
    print(row)

['ID', 'Name', 'Surname', 'ItemsBought', 'Spent', 'Saved']
(100000119, 'Vanessa', 'Lyman', 8, 732.9, 7.329000000000001)


In [130]:
cursor.execute('''SELECT customer.CustomerID as ID, customer.Name as Name, customer.Surname as Surname,  strftime('%Y-%m-%d', transactions.Timestamp) as BullishDay, COUNT(transactions.ItemID) as ItemsBought, strftime('%w', transactions.Timestamp) as DayOfWeek FROM transactions
                  INNER JOIN item
                    ON item.ItemID = transactions.ItemID
                  INNER JOIN customer
                    ON customer.CustomerID = transactions.CustomerID
                  WHERE Name = 'Vanessa' AND Surname = 'Lyman'
                  GROUP BY BullishDay
                  ORDER BY BullishDay DESC
                  LIMIT 1
               ''')

# Fetch the column names and print them
columns = [description[0] for description in cursor.description]
print(columns)

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results of people who bought more in quantity
for row in results:
    print(row)

['ID', 'Name', 'Surname', 'BullishDay', 'ItemsBought', 'DayOfWeek']
(100000119, 'Vanessa', 'Lyman', '2016-12-13', 3, '2')
